In [2]:
!pip install torch
!pip install torchvision

    100% |████████████████████████████████| 591.8MB 25kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x62756000 @  0x7fd75ac142a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 4.1MB/s 
    100% |████████████████████████████████| 2.0MB 9.8MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [6]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110377 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive -o nonempty
import os
os.chdir('drive/Alzheimer_Dataset')

In [0]:
from PIL import Image
def register_extension(id, extension): Image.EXTENSION[extension.lower()] = id.upper()
Image.register_extension = register_extension
def register_extensions(id, extensions): 
  for extension in extensions: register_extension(id, extension)
Image.register_extensions = register_extensions

In [23]:
!pip install Pillow==4.0.0
!pip install PIL
!pip install image

    100% |████████████████████████████████| 5.6MB 6.6MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [0]:
train_data_dir = 'dataset/Train/'
validation_data_dir = 'dataset/Test/'

transform = transforms.Compose([transforms.ToTensor()])
train_datasets = torchvision.datasets.ImageFolder(train_data_dir, transform= transform)
test_datasets= torchvision.datasets.ImageFolder(validation_data_dir, transform= transform)

trainLoader = torch.utils.data.DataLoader(train_datasets, batch_size = 1, shuffle= True, num_workers = 2)
testLoader = torch.utils.data.DataLoader(test_datasets, batch_size = 1, shuffle= False, num_workers = 2)

In [0]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = torch.nn.Conv2d(3, 6, 5)
    self.pool = torch.nn.MaxPool2d(2, 2)
    self.conv2 = torch.nn.Conv2d(6, 16, 5)
    self.fc1 = torch.nn.Linear(32144, 120)
    self.fc2 = torch.nn.Linear(120, 84)
    self.fc3 = torch.nn.Linear(84, 10)
  
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    # print(x.size())
    x = x.view(-1, 32144)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [51]:
model = CNN()
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr= 0.1, momentum = 0.9)
num_epochs = 2

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=32144, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


# Training routine

In [52]:
for epoch in range(num_epochs):
  running_loss=0.0
  for i, (img, target) in enumerate(trainLoader):
      img = Variable(img)
      target= Variable(target)
      model.zero_grad()
      
      outputs = model(img)
      loss = criterion(outputs, target)
      loss.backward()
      running_loss += loss.item()
      optimizer.step()
      if i % 200 == 199:    # print every 2000 mini-batches
          print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 200))
          running_loss = 0.0
        

[1,   200] loss: 1.369
[1,   400] loss: 1.313
[1,   600] loss: 1.119
[1,   800] loss: 1.243
[1,  1200] loss: 1.259
[1,  1400] loss: 1.194
[1,  1600] loss: 1.185
[1,  1800] loss: 1.131
[1,  2000] loss: 1.218
[1,  2200] loss: 1.290
[1,  2400] loss: 1.319
[1,  2600] loss: 1.184
[1,  2800] loss: 1.187
[1,  3000] loss: 1.267
[1,  3200] loss: 1.223
[1,  3400] loss: 1.126
[1,  3600] loss: 1.258
[1,  3800] loss: 1.145
[1,  4000] loss: 1.221
[1,  4200] loss: 1.245
[1,  4400] loss: 1.196
[1,  4800] loss: 1.365
[1,  5000] loss: 1.146
[2,   200] loss: 1.246
[2,   600] loss: 1.146
[2,   800] loss: 1.253
[2,  1000] loss: 1.180
[2,  1200] loss: 1.273
[2,  1400] loss: 1.199
[2,  1600] loss: 1.129
[2,  1800] loss: 1.182
[2,  2000] loss: 1.191
[2,  2200] loss: 1.216
[2,  2400] loss: 1.224
[2,  2600] loss: 1.294
[2,  2800] loss: 1.135
[2,  3000] loss: 1.200
[2,  3200] loss: 1.204
[2,  3400] loss: 1.332
[2,  3600] loss: 1.202
[2,  3800] loss: 1.179
[2,  4000] loss: 1.156
[2,  4200] loss: 1.219
[2,  4400] 

# Testing the network

In [56]:
total = 0
correct = 0
for data in testLoader:
  img, labels = data
  img = Variable(img)
  labels = Variable(labels)
  outputs= model(img)
  _, predicted = torch.max(outputs.data , 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 50 %
